## DES (Encryption & Decryption)

In [1]:
pip install pycryptodome

                                              0.0/1.8 MB ? eta -:--:--
                                              0.0/1.8 MB 1.3 MB/s eta 0:00:02
     -                                        0.1/1.8 MB 812.7 kB/s eta 0:00:03
     --                                       0.1/1.8 MB 737.3 kB/s eta 0:00:03
     --                                       0.1/1.8 MB 722.1 kB/s eta 0:00:03
     --                                       0.1/1.8 MB 722.1 kB/s eta 0:00:03
     --                                       0.1/1.8 MB 722.1 kB/s eta 0:00:03
     -----                                    0.2/1.8 MB 686.8 kB/s eta 0:00:03
     -----                                    0.3/1.8 MB 714.4 kB/s eta 0:00:03
     -------                                  0.3/1.8 MB 728.0 kB/s eta 0:00:03
     -------                                  0.3/1.8 MB 723.4 kB/s eta 0:00:03
     -------                                  0.3/1.8 MB 723.4 kB/s eta 0:00:03
     ---------                                0.5/1

In [3]:
from Crypto.Cipher import DES
from Crypto.Util.Padding import pad, unpad
from Crypto.Random import get_random_bytes

def des_encrypt(plaintext, key):
    cipher = DES.new(key, DES.MODE_ECB)
    
    padded_text = pad(plaintext.encode(), DES.block_size)
    
    ciphertext = cipher.encrypt(padded_text)
    return ciphertext

def des_decrypt(ciphertext, key):
    cipher = DES.new(key, DES.MODE_ECB)
    
    decrypted_padded_text = cipher.decrypt(ciphertext)
    plaintext = unpad(decrypted_padded_text, DES.block_size).decode()
    return plaintext


if __name__ == "__main__":
    key = get_random_bytes(8) 
    plaintext = "Hello, Jeyadev!"

    print(f"Original Text: {plaintext}")
    
    encrypted_text = des_encrypt(plaintext, key)
    print(f"Encrypted Text: {encrypted_text}")
    
    decrypted_text = des_decrypt(encrypted_text, key)
    print(f"Decrypted Text: {decrypted_text}")


Original Text: Hello, Jeyadev!
Encrypted Text: b'\x92A\x07\xfb\x8d\x8a,\xc2\xa4xw\x8bo\xe8K\x93'
Decrypted Text: Hello, Jeyadev!


In [2]:
pip install tinyec

## Hill Cipher 

In [12]:
import numpy as np
from math import gcd

def mod_inverse(a, m):
    a = a % m
    for x in range(1, m):
        if (a * x) % m == 1:
            return x
    return None

def matrix_mod_inverse_2x2(matrix, mod):
    determinant = int(np.round(np.linalg.det(matrix)))
    determinant = determinant % mod
    if gcd(determinant, mod) != 1:
        raise ValueError("Matrix is not invertible under mod 26")
    
    determinant_inv = mod_inverse(determinant, mod)
    if determinant_inv is None:
        raise ValueError("Matrix is not invertible under mod 26")

    adjugate = np.array([[matrix[1][1], -matrix[0][1]],
                         [-matrix[1][0], matrix[0][0]]])

    matrix_inv = (determinant_inv * adjugate) % mod
    matrix_inv = matrix_inv.astype(int)

    return matrix_inv

def create_key_matrix_2x2(key):
    """Create a 2x2 key matrix from the key."""
    key_matrix = np.array([ord(key[0]) - 65, ord(key[1]) - 65,
                           ord(key[2]) - 65, ord(key[3]) - 65]).reshape(2, 2)
    return key_matrix

def text_to_numbers(text):
    """Convert text to numerical representation (A=0, ..., Z=25)."""
    return [ord(char) - 65 for char in text]

def numbers_to_text(numbers):
    """Convert numerical representation back to text."""
    return ''.join(chr(num + 65) for num in numbers)

def encrypt_hill_cipher(plaintext, key):
    """Encrypt the plaintext using the Hill cipher."""
    key_matrix = create_key_matrix_2x2(key)
    plaintext_numbers = text_to_numbers(plaintext)

    if len(plaintext_numbers) % 2 != 0:
        plaintext_numbers.append(ord('X') - 65) 

    ciphertext_numbers = []
    for i in range(0, len(plaintext_numbers), 2):
        pair = np.array(plaintext_numbers[i:i+2]).reshape(2, 1)
        encrypted_pair = np.dot(key_matrix, pair) % 26
        ciphertext_numbers.extend(encrypted_pair.flatten())

    return numbers_to_text(ciphertext_numbers)

def decrypt_hill_cipher(ciphertext, key):
    """Decrypt the ciphertext using the Hill cipher."""
    key_matrix = create_key_matrix_2x2(key)
    key_matrix_inv = matrix_mod_inverse_2x2(key_matrix, 26)
    ciphertext_numbers = text_to_numbers(ciphertext)

    plaintext_numbers = []
    for i in range(0, len(ciphertext_numbers), 2):
        pair = np.array(ciphertext_numbers[i:i+2]).reshape(2, 1)
        decrypted_pair = np.dot(key_matrix_inv, pair) % 26
        plaintext_numbers.extend(decrypted_pair.flatten())

    return numbers_to_text(plaintext_numbers).rstrip('X')

key = "FJDE"
plaintext = ("As cybersecurity Leaders we have to create our message of 
             influence because security is a culture and you need the business to take place"
             .replace(" ", "").upper())

# Encrypt the plaintext
encrypted_text = encrypt_hill_cipher(plaintext, key)
print(f"Encrypted Text: {encrypted_text}")

# Decrypt the ciphertext
decrypted_text = decrypt_hill_cipher(encrypted_text, key)
print(f"Decrypted Text: {decrypted_text}")


Encrypted Text: GUSYPTNTMUTYDWLMUMZZNTQEJVLBNJHWUMBVQSLVAGMCOILKBYUHGYFVDQMUYCWSWSIIBFZXUSSIRAPHRPNAXBQSXDVYCHDQCCBYAGBAHOMOZUDHUW
Decrypted Text: ASCYBERSECURITYLEADERSWEHAVETOCREATEOURMESSAGEOFINFLUENCEBECAUSESECURITYISACULTUREANDYOUNEEDTHEBUSINESSTOTAKEPLACE


In [3]:
import numpy as np

def create_key_matrix(key):
    return np.array([ord(c) - 65 for c in key]).reshape(2, 2)

def text_to_numbers(text):
    return [ord(char) - 65 for char in text]

def numbers_to_text(numbers):
    return ''.join(chr(num + 65) for num in numbers)

def encrypt_hill_cipher(plaintext, key):
    key_matrix = create_key_matrix(key)
    plaintext = ''.join(plaintext.split()).upper()
    plaintext_numbers = text_to_numbers(plaintext)
    if len(plaintext_numbers) % 2 != 0:
        plaintext_numbers.append(23)  # Padding with 'X'

    ciphertext_numbers = []
    for i in range(0, len(plaintext_numbers), 2):
        pair = np.dot(key_matrix, plaintext_numbers[i:i+2]) % 26
        ciphertext_numbers.extend(pair)

    return numbers_to_text(ciphertext_numbers)

key = "FJDE"
plaintext = "As cybersecurity leaders we have to create our message of influence because security is a culture and you need the business to take place"

encrypted_text = encrypt_hill_cipher(plaintext, key)
print(f"Encrypted Text: {encrypted_text}")


Encrypted Text: GUSYPTNTMUTYDWLMUMZZNTQEJVLBNJHWUMBVQSLVAGMCOILKBYUHGYFVDQMUYCWSWSIIBFZXUSSIRAPHRPNAXBQSXDVYCHDQCCBYAGBAHOMOZUDHUW


In [1]:
pip install tinyec matplotlib


Note: you may need to restart the kernel to use updated packages.


## DES

In [3]:
initial_permutation_table = [
    58, 50, 42, 34, 26, 18, 10, 2,
    60, 52, 44, 36, 28, 20, 12, 4,
    62, 54, 46, 38, 30, 22, 14, 6,
    64, 56, 48, 40, 32, 24, 16, 8,
    57, 49, 41, 33, 25, 17, 9, 1,
    59, 51, 43, 35, 27, 19, 11, 3,
    61, 53, 45, 37, 29, 21, 13, 5,
    63, 55, 47, 39, 31, 23, 15, 7
]

expansion_table = [
    32, 1, 2, 3, 4, 5, 4, 5, 6, 7, 8, 9,
    8, 9, 10, 11, 12, 13, 12, 13, 14, 15, 16, 17,
    16, 17, 18, 19, 20, 21, 20, 21, 22, 23, 24, 25,
    24, 25, 26, 27, 28, 29, 28, 29, 30, 31, 32, 1
]

PC1_table = [
    57, 49, 41, 33, 25, 17, 9, 1,
    58, 50, 42, 34, 26, 18, 10, 2,
    59, 51, 43, 35, 27, 19, 11, 3,
    60, 52, 44, 36, 63, 55, 47, 39,
    31, 23, 15, 7, 62, 54, 46, 38,
    30, 22, 14, 6, 61, 53, 45, 37,
    29, 21, 13, 5, 28, 20, 12, 4
]

shift_schedule = [1, 1, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 1]

PC2_table = [
    14, 17, 11, 24, 1, 5, 3, 28, 15, 6, 21, 10,
    23, 19, 12, 4, 26, 8, 16, 7, 27, 20, 13, 2,
    41, 52, 31, 37, 47, 55, 30, 40, 51, 45, 33, 48,
    44, 49, 39, 56, 34, 53, 46, 42, 50, 36, 29, 32
]

sbox = [
    # S1
    [[14, 4, 13, 1, 2, 15, 11, 8, 3, 10, 6, 12, 5, 9, 0, 7],
     [0, 15, 7, 4, 14, 2, 13, 1, 10, 6, 12, 11, 9, 5, 3, 8],
     [4, 1, 14, 8, 13, 6, 2, 11, 15, 12, 9, 7, 3, 10, 5, 0],
     [15, 12, 8, 2, 4, 9, 1, 7, 5, 11, 3, 14, 10, 0, 6, 13]],
    
    # S2
    [[15, 1, 8, 14, 6, 11, 3, 4, 9, 7, 2, 13, 12, 0, 5, 10],
     [3, 13, 4, 7, 15, 2, 8, 14, 12, 0, 1, 10, 6, 9, 11, 5],
     [0, 14, 7, 11, 10, 4, 13, 1, 5, 8, 12, 6, 9, 3, 2, 15],
     [13, 8, 10, 1, 3, 15, 4, 2, 11, 6, 7, 12, 0, 5, 14, 9]],
    
    # S3
    [[10, 0, 9, 14, 6, 3, 15, 5, 1, 13, 12, 7, 11, 4, 2, 8],
     [13, 7, 0, 9, 3, 4, 6, 10, 2, 8, 5, 14, 12, 11, 15, 1],
     [13, 6, 4, 9, 8, 15, 3, 0, 11, 1, 2, 12, 5, 10, 14, 7],
     [1, 10, 13, 0, 6, 9, 8, 7, 4, 15, 14, 3, 11, 5, 2, 12]],
    
    # S4
    [[7, 13, 14, 3, 0, 6, 9, 10, 1, 2, 8, 5, 11, 12, 4, 15],
     [13, 8, 11, 5, 6, 15, 0, 3, 4, 7, 2, 12, 1, 10, 14, 9],
     [10, 6, 9, 0, 12, 11, 7, 13, 15, 1, 3, 14, 5, 2, 8, 4],
     [3, 15, 0, 6, 10, 1, 13, 8, 9, 4, 5, 11, 12, 7, 2, 14]],
    
    # S5
    [[2, 12, 4, 1, 7, 10, 11, 6, 8, 5, 3, 15, 13, 0, 14, 9],
     [14, 11, 2, 12, 4, 7, 13, 1, 5, 0, 15, 10, 3, 9, 8, 6],
     [4, 2, 1, 11, 10, 13, 7, 8, 15, 9, 12, 5, 6, 3, 0, 14],
     [11, 8, 12, 7, 1, 14, 2, 13, 6, 15, 0, 9, 10, 4, 5, 3]],
    
    # S6
    [[12, 1, 10, 15, 9, 2, 6, 8, 0, 13, 3, 4, 14, 7, 5, 11],
     [10, 15, 4, 2, 7, 12, 9, 5, 6, 1, 13, 14, 0, 11, 3, 8],
     [9, 14, 15, 5, 2, 8, 12, 3, 7, 0, 4, 10, 1, 13, 11, 6],
     [4, 3, 2, 12, 9, 5, 15, 10, 11, 14, 1, 7, 6, 0, 8, 13]],
    
    # S7
    [[4, 11, 2, 14, 15, 0, 8, 13, 3, 12, 9, 7, 5, 10, 6, 1],
     [13, 0, 11, 7, 4, 9, 1, 10, 14, 3, 5, 12, 2, 15, 8, 6],
     [1, 4, 11, 13, 12, 3, 7, 14, 10, 15, 6, 8, 0, 5, 9, 2],
     [6, 11, 13, 8, 1, 4, 10, 7, 9, 5, 0, 15, 14, 2, 3, 12]],
    
    # S8
    [[13, 2, 8, 4, 6, 15, 11, 1, 10, 9, 3, 14, 5, 0, 12, 7],
     [1, 15, 13, 8, 10, 3, 7, 4, 12, 5, 6, 11, 0, 14, 9, 2],
     [7, 11, 4, 1, 9, 12, 14, 2, 0, 6, 10, 13, 15, 3, 5, 8],
     [2, 1, 14, 7, 4, 10, 8, 13, 15, 12, 9, 0, 3, 5, 6, 11]]
]

straight_permutation_table = [
    16, 7, 20, 21, 29, 12, 28, 17, 
    1, 15, 23, 26, 5, 18, 31, 10, 
    2, 8, 24, 14, 32, 27, 3, 9, 
    19, 13, 30, 6, 22, 11, 4, 25
]

def initial_permutation(input):
    permuted_data = 0
    for i in range(64):
        bit_position = initial_permutation_table[i] - 1
        bit = (input >> (64 - bit_position)) & 0x01
        permuted_data |= (bit << (63 - i))
    return permuted_data

def divide_64_bits(input):
    left = (input >> 32) & 0xFFFFFFFF
    right = input & 0xFFFFFFFF
    return left, right

def expansion_box(input):
    expanded_output = 0
    for i in range(48):
        bit_position = expansion_table[i] - 1
        expanded_output |= ((input >> (32 - bit_position)) & 0x01) << (47 - i)
    return expanded_output

def xor_48_bits(value1, value2):
    return value1 ^ value2

def sbox_substitution(input_chunk, sbox_number):
    row = ((input_chunk >> 5) & 0x01) << 1 | (input_chunk & 0x01)
    column = (input_chunk >> 1) & 0x0F
    return sbox[sbox_number][row][column]

def straight_permutation(input):
    permuted_data = 0
    for i in range(32):
        bit_position = straight_permutation_table[i] - 1
        bit = (input >> (32 - bit_position)) & 0x01
        permuted_data |= (bit << (31 - i))
    return permuted_data

def xor_operation(left, right):
    return left ^ right

def concatenate(xor_result, right):
    return (xor_result << 32) | right

def key_schedule(key):
    keys = []
    
    key = 0
    for i in range(56):
        bit_position = PC1_table[i] - 1
        key |= ((key >> (64 - bit_position)) & 0x01) << (55 - i)
    
    left = (key >> 28) & 0xFFFFFFF
    right = key & 0xFFFFFFF

    for round in range(16):
        left = ((left << shift_schedule[round]) | (left >> (28 - shift_schedule[round]))) & 0xFFFFFFF
        right = ((right << shift_schedule[round]) | (right >> (28 - shift_schedule[round]))) & 0xFFFFFFF
        
        combined_key = (left << 28) | right
        
        subkey = 0
        for i in range(48):
            bit_position = PC2_table[i] - 1
            subkey |= ((combined_key >> (56 - bit_position)) & 0x01) << (47 - i)
        keys.append(subkey)

    return keys

def des_encrypt(input_data, key):
    encrypted_data = initial_permutation(input_data)
    subkeys = key_schedule(key)

    for round in range(16):
        left, right = divide_64_bits(encrypted_data)

        expanded_right = expansion_box(right)
        xor_result = xor_48_bits(expanded_right, subkeys[round])

        sbox_output = 0
        for i in range(0, 48, 6):
            chunk = (xor_result >> (48 - (i + 6))) & 0x3F
            sbox_output <<= 4
            sbox_output |= sbox_substitution(chunk, i // 6)

        straight_permuted_data = straight_permutation(sbox_output)
        left = xor_operation(left, straight_permuted_data)

        encrypted_data = concatenate(left, right)
        print(f"Round {round + 1}: Encrypted Result: 0x{encrypted_data:016X}")

    final_data = 0
    for i in range(64):
        bit_position = initial_permutation_table[i] - 1
        bit = (encrypted_data >> (64 - bit_position)) & 0x01
        final_data |= (bit << (63 - i))

    return final_data

input_data = 0x123456789FFFFFF
key = 0x0ABCDEF0123456789  
final_encrypted_data = des_encrypt(input_data, key)
print(f"Final Encrypted Result: 0x{final_encrypted_data:016X}")

Round 1: Encrypted Result: 0x19B0BD53FEECE0EC
Round 2: Encrypted Result: 0xF0EAF0EAFEECE0EC
Round 3: Encrypted Result: 0x19B0BD53FEECE0EC
Round 4: Encrypted Result: 0xF0EAF0EAFEECE0EC
Round 5: Encrypted Result: 0x19B0BD53FEECE0EC
Round 6: Encrypted Result: 0xF0EAF0EAFEECE0EC
Round 7: Encrypted Result: 0x19B0BD53FEECE0EC
Round 8: Encrypted Result: 0xF0EAF0EAFEECE0EC
Round 9: Encrypted Result: 0x19B0BD53FEECE0EC
Round 10: Encrypted Result: 0xF0EAF0EAFEECE0EC
Round 11: Encrypted Result: 0x19B0BD53FEECE0EC
Round 12: Encrypted Result: 0xF0EAF0EAFEECE0EC
Round 13: Encrypted Result: 0x19B0BD53FEECE0EC
Round 14: Encrypted Result: 0xF0EAF0EAFEECE0EC
Round 15: Encrypted Result: 0x19B0BD53FEECE0EC
Round 16: Encrypted Result: 0xF0EAF0EAFEECE0EC
Final Encrypted Result: 0xFFFFBA1A00FF15B0
